In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

import os

import math

from tensorflow.keras.utils import Sequence
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from tensorflow.keras.models import load_model

pd.set_option('display.max_columns', 100)


In [3]:
# Load the data and preprocess it into train, validation, and test sets 
df = pd.read_csv('../../data/latest_hour_final_eth.csv')


In [4]:
df.tail(100)

,Unnamed: 0,datetime,open,high,low,close,volume,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore,ao,kama,ppo,pvo,roc,rsi,stoch,tsi,uo,wri,accdist,cmf,emv,fi,mfi,nvi,obv,smaemv,vpt,vwap,atr,ulcer,bbh,bbl,bbhi,bbli,bbmavg,bb_pb,bb_wb,dchb,dclb,dcmb,dcpb,dcwb,kchb,kclb,kchbi,kclbi,kcmb,kcpb,kcwb,aroon_up,aroon_down,cci,dpo,kst,kst_sig,ema,ichimoku_a,ichimoku_b,ichimoku_base_line,ichimoku_conversion_line,macd,macd_diff,macd_signal,mi,sma,wma,stc,vi_pos,vi_neg,DATE
19330,20139,1682010000000,1937.81,1949.41,1931.35,1935.74,30088.8095,1682013599999,5.836686e+07,44849,14604.2850,2.833133e+07,0,-66.582583,1993.067316,-0.697812,9.655065,3.042175,45.830656,54.809951,-26.393605,47.268793,-45.190049,2.429009e+07,-0.008045,3.250208e+05,-147844.392689,40.500272,59648.330503,171159.18557,-217082.606133,-264.466848,1952.823856,16.969277,3.290053,1973.557653,1929.002347,0.0,0.0,1951.2800,0.151220,2.283389,2125.0,1914.43,2019.715,0.101202,10.422268,1971.705778,1934.197778,0.0,0.0,1952.951778,0.041117,1.920580,20.833333,85.000000,-105.251884,-44.965500,-40.258680,-31.995065,2006.536028,1950.8175,2013.315,1952.925,1948.710,-53.156884,-5.987727,-47.169158,121.773931,2020.385500,2011.614620,1.213624e-13,0.930798,1.064404,2023-04-20
19331,20140,1682013600000,1935.73,1944.66,1921.14,1933.27,36163.6507,1682017199999,6.984320e+07,50282,17955.4703,3.467562e+07,0,-65.774750,1992.694608,-0.722816,9.587030,3.147342,45.698154,54.267583,-26.508247,49.429585,-45.732417,2.429123e+07,-0.019850,-4.864819e+05,-144068.897498,38.579834,59648.330503,134995.53487,-241719.760081,-78.441198,1951.835332,17.078456,3.329217,1972.162773,1927.124227,0.0,0.0,1949.6435,0.136456,2.310091,2125.0,1914.43,2019.715,0.089471,10.437024,1970.972889,1933.846222,0.0,1.0,1952.409556,-0.015520,1.901582,20.000000,100.000000,-135.889694,-36.089167,-42.611959,-34.032534,2005.928011,1949.9600,2011.195,1951.210,1948.710,-54.064539,-6.033459,-48.031081,122.001828,2017.529167,2011.423635,6.068118e-14,0.917229,1.063452,2023-04-20
19332,20141,1682017200000,1933.27,1933.75,1916.06,1927.70,32212.7863,1682020799999,6.202306e+07,48104,14878.5077,2.864587e+07,0,-65.917083,1992.089131,-0.757150,8.474590,2.959477,45.399676,53.044509,-26.722600,44.966720,-46.955491,2.430141e+07,-0.029731,-4.390541e+05,-146349.950542,37.139607,59476.475976,102782.74857,-252479.803421,-138.953927,1949.871885,17.088648,3.370073,1972.845395,1925.838605,0.0,0.0,1949.3420,0.039598,2.411418,2125.0,1914.43,2019.715,0.063019,10.451925,1969.654889,1931.509556,0.0,1.0,1950.582222,-0.099870,1.955587,19.166667,100.000000,-170.273228,-36.822833,-44.903253,-36.114777,2005.278816,1949.9600,2011.195,1951.210,1948.710,-55.109237,-6.193387,-48.915850,122.131314,2014.652833,2011.201280,3.034059e-14,0.902092,1.088433,2023-04-20
19333,20142,1682020800000,1927.70,1944.39,1923.06,1938.07,21696.9007,1682024399999,4.198961e+07,32688,11732.0850,2.270312e+07,0,-65.414917,1991.583875,-0.733092,4.918905,3.768847,46.061091,55.321578,-26.561026,46.892039,-44.678422,2.431025e+07,-0.016351,8.670851e+05,-122392.091781,38.259111,59796.427761,124479.64927,-257154.816871,23.908589,1949.029981,17.159338,3.405435,1972.904156,1924.917844,0.0,0.0,1948.9110,0.274081,2.462212,2125.0,1914.43,2019.715,0.112267,10.465648,1969.050000,1929.671333,0.0,0.0,1949.360667,0.213280,2.020081,18.333333,99.166667,-95.866021,-27.491167,-46.877135,-38.196439,2004.721066,1949.9600,2011.195,1951.210,1948.710,-55.513986,-5.773369,-49.740617,122.210302,2012.011167,2011.036092,1.517029e-14,0.895813,1.107155,2023-04-20
19334,20143,1682024400000,1938.07,1945.92,1935.84,1942.72,11326.2713,1682027999999,2.197366e+07,21639,5947.1311,1.153759e+07,0,-63.239333,1991.304130,-0.686869,-0.856486,3.877105,46.354934,56.342636,-26.255910,49.128567,-43.657364,2.431439e+07,-0.002777,6.367709e+05,-111097.946405,37.994695,59939.896980,135805.92057,-243515.756902,143.892834,1948.507092,17.041349,3.438915,1972.613543,1924.375457,0.0,0.0,1948.4945,0.380292,2.475659,2125.0,1914.43,2019.715,0.1

In [129]:
time = 1681334000000-(3600*1000*814) # 479
#time = 192
df_reduced_last_time = df.loc[df['datetime'] >= time]
print(len(df_reduced_last_time))

df_reduced_last_time.tail(10)

1055


,Unnamed: 0,datetime,open,high,low,close,volume,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore,ao,kama,ppo,pvo,roc,rsi,stoch,tsi,uo,wri,accdist,cmf,emv,fi,mfi,nvi,obv,smaemv,vpt,vwap,atr,ulcer,bbh,bbl,bbhi,bbli,bbmavg,bb_pb,bb_wb,dchb,dclb,dcmb,dcpb,dcwb,kchb,kclb,kchbi,kclbi,kcmb,kcpb,kcwb,aroon_up,aroon_down,cci,dpo,kst,kst_sig,ema,ichimoku_a,ichimoku_b,ichimoku_base_line,ichimoku_conversion_line,macd,macd_diff,macd_signal,mi,sma,wma,stc,vi_pos,vi_neg,DATE
19420,20229,1682330400000,1839.13,1849.14,1839.12,1844.80,10670.9724,1682333999999,1.967811e+07,19489,5291.2552,9.757174e+06,0,-5.542083,1923.673361,-0.312799,9.132119,-2.384304,43.852655,21.656942,-14.883525,45.487688,-65.158868,2.417948e+07,-0.004613,4.676200e+05,-30552.574632,51.338055,59653.586644,-92507.87183,-43630.760466,77.902301,1853.271786,14.255620,7.098145,1877.950706,1833.937294,0.0,0.0,1855.9440,0.246804,2.371484,1890.0,1827.23,1858.615,0.279911,3.372956,1870.573333,1841.244000,0.0,0.0,1855.908667,0.121244,1.580322,2.500000,98.333333,-82.349604,0.820833,-63.791327,-66.471466,1936.261838,1854.5275,1858.615,1858.615,1850.440,-45.554265,1.836004,-47.390269,118.505766,1860.979167,1965.874702,12.487031,0.945018,1.040569,2023-04-24
19421,20230,1682334000000,1844.79,1855.57,1844.39,1855.26,10934.2568,1682337599999,2.022543e+07,20108,6174.6507,1.142251e+07,0,-8.544750,1923.561865,-0.272353,4.999632,-1.741917,44.631977,24.418513,-14.498337,50.779331,-62.862036,2.418981e+07,-0.001218,5.981476e+05,-21202.581035,51.072981,59653.586644,-81573.61503,-47520.061165,94.953731,1853.571041,14.204359,7.135697,1877.695057,1833.787943,0.0,0.0,1855.7415,0.489034,2.366015,1890.0,1827.23,1858.615,0.446551,3.372760,1870.722889,1841.805556,0.0,0.0,1856.264222,0.465273,1.557824,1.666667,97.500000,-26.670709,3.232833,-63.098899,-65.778028,1935.589623,1853.6675,1858.615,1858.615,1848.720,-44.944727,2.139849,-47.084576,118.269877,1861.087167,1965.236320,31.244502,0.962397,1.029894,2023-04-24
19422,20231,1682337600000,1855.26,1856.61,1846.84,1847.90,15041.1844,1682341199999,2.786218e+07,20126,5841.1472,1.081669e+07,0,-10.279167,1923.320448,-0.269258,3.508395,-2.041963,44.196708,22.475381,-14.392338,50.531094,-64.478163,2.417803e+07,-0.024204,1.133465e+05,-26976.809173,49.548659,59653.586644,-96614.79943,-52370.460953,2.327274,1853.375889,14.130453,7.175054,1876.607344,1833.127656,0.0,0.0,1854.8675,0.339753,2.344086,1890.0,1827.23,1858.615,0.329297,3.372743,1870.365111,1841.709111,0.0,0.0,1856.037111,0.216042,1.543935,0.833333,96.666667,-32.209528,5.893333,-62.382307,-65.090816,1934.861909,1853.2150,1858.615,1858.615,1847.815,-44.689197,2.095957,-46.785154,118.092610,1861.096667,1964.556910,53.123731,0.955265,1.038502,2023-04-24
19423,20232,1682341200000,1847.90,1871.86,1846.98,1865.10,23560.3775,1682344799999,4.387613e+07,36649,12159.6383,2.263974e+07,0,-10.961583,1923.255637,-0.189704,6.199443,0.012333,45.450392,27.016395,-13.545758,55.827781,-60.701346,2.418879e+07,-0.017769,8.125999e+05,908.049031,50.125063,59653.586644,-73054.42193,-43962.337600,159.626899,1853.735040,14.309612,7.208046,1877.337219,1833.048781,0.0,0.0,1855.1930,0.723693,2.387269,1890.0,1827.23,1858.615,0.603314,3.372468,1870.814444,1841.529111,0.0,0.0,1856.171778,0.804870,1.577728,1.666667,95.833333,46.241870,7.071667,-61.524784,-64.384532,1934.282972,1854.0800,1858.615,1858.615,1849.545,-43.581067,2.803575,-46.384643,118.014233,1861.248333,1963.972619,70.313592,0.958818,1.011449,2023-04-24
19424,20233,1682344800000,1865.10,1876.19,1843.07,1855.24,31345.4489,1682348399999,5.828815e+07,45052,15016.3794,2.792689e+07,0,-11.561250,1923.087498,-0.167657,11.270208,-0.405306,44.867693,24.413232,-13.112717,48.995423,-62.866428,2.418048e+07,-0.040532,2.218887e+04,-19090.284852,47.811286,59653.586644,-104399.87083,-52637.099814,53.586548,1853.962420,14.623119,7.243410,1877.421547,1833.811453,0.0,0.0,1855.6165,0.491367,2.350167,1890.0,1827.23,1858.615,0.446232,3.372456,1872.321111,1839.477111,0.0,0.0,1855.899111,0

In [130]:

df_reduced = df_reduced_last_time.drop(columns=['Unnamed: 0','datetime','DATE'])

scaler = MinMaxScaler()

# Fit the scaler on your dataframe (let's say it's called df)
df_normalized = scaler.fit_transform(df_reduced)

# Convert the normalized data back to a dataframe
df_normalized = pd.DataFrame(df_normalized, columns=df_reduced.columns)

# Convert the DataFrame to a NumPy array
raw_data = df_normalized.values
close = df_normalized['close'].values
close = close.reshape((len(close),1))

# Display the NumPy array
print(type(raw_data))
print(raw_data.shape)
print(close.shape)


<class 'numpy.ndarray'>
(1055, 72)
(1055, 1)


In [131]:
# Set Parameters

# delay: time in future that will be predicted
delay = 7*24 # one Hour
# delay = 3 # 3hour

# sampling rate: period between timesteps within the sequence
# Sequence with rate=1 : t1,t2...tn
# Sequence with rate=3 : t1,t3...tn*3
sampling_rate = 6

# sequence length: sequence lenght of each sample 
sequence_length = 5 * 24


# sequence_stride: period between sequences
# First sequence starts at t0
# Second sequence will start at t1 with sequence_stride=1 or at t5 with sequence_stride=5
sequence_stride = 1

#batch_size: Number of timeseries samples in each batch (except maybe the last one). 
#If None, the data will not be batched (the dataset will yield individual samples).
# Huge impact in performance.
# Tip, should be multiple of 8
batch_size = 32

# Understanding our parameters
msg = f"The timeseries will consist of batches containing {batch_size} sequences of {sequence_length} samples."

msg += f"\nFinally our target is {delay} timesteps in the future, and will have data from {sequence_length * sampling_rate} timesteps in the past"
print(msg)

   

keras_dataset = keras.preprocessing.timeseries_dataset_from_array(
                    raw_data[:-delay],
                    targets=close[delay:],
                    sampling_rate=sampling_rate,
                    sequence_stride=sequence_stride,
                    sequence_length=sequence_length,
                    shuffle=False, # Shouldn't the shuffle be set to 0?
                    seed=33,
                    batch_size=batch_size,
                    start_index=0)

print("Done Data")

The timeseries will consist of batches containing 32 sequences of 120 samples.
Finally our target is 168 timesteps in the future, and will have data from 720 timesteps in the past
Done Data


In [132]:
# Load the model from the .h5 file
modelo_7d =  load_model('lstm_model_7d_ahead.h5')
# tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
# tf.config.run_functions_eagerly(True)

2023-05-20 17:57:44.087867: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-20 17:57:44.089898: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-20 17:57:44.091326: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [133]:

test_pred = modelo_7d.predict(keras_dataset)


2023-05-20 17:57:44.339430: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [887,72]
	 [[{{node Placeholder/_0}}]]
2023-05-20 17:57:44.339972: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_8' with dtype int32 and shape [168]
	 [[{{node Placeholder/_8}}]]


6/6 [==============================] - 0s 11ms/step


2023-05-20 17:57:44.630562: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-20 17:57:44.632583: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-20 17:57:44.634192: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [134]:
# Assume 'y_normalized' contains the predicted values for the 'target' column in normalized form
y_min = df_reduced['close'].min()
y_max = df_reduced['close'].max()

test_pred = test_pred * (y_max - y_min) + y_min


In [135]:
len(test_pred)

168

In [116]:
test_pred

array([[1920.3916],
       [1921.4205],
       [1922.8794],
       [1925.3147],
       [1928.8665],
       [1927.5   ],
       [1929.03  ]], dtype=float32)

In [137]:
predicted_prices = pd.DataFrame(test_pred, columns=['close_price'])
predicted_prices

,close_price
0,1687.062866
1,1688.920654
2,1685.669556
3,1687.740479
4,1684.156860
...,...
163,1796.726196
164,1800.311646
165,1804.974121
166,1803.641113


In [145]:
# Function to add incremental number starting from a specific value
def add_incremental_number(row, increment, starting_number):
    return starting_number + row.name * increment


# Add incremental number to each row starting from 90
increment = 30
starting_number = 90
predicted_prices['Number'] = df.apply(add_incremental_number, args=(increment, starting_number), axis=1)


In [146]:
predicted_prices

,close_price,Number
0,1687.062866,90
1,1688.920654,120
2,1685.669556,150
3,1687.740479,180
4,1684.156860,210
...,...,...
163,1796.726196,4980
164,1800.311646,5010
165,1804.974121,5040
166,1803.641113,5070
